In [44]:
import pandas as pd
import numpy as np
%matplotlib inline  

In [45]:
pd.options.display.max_columns= 100

In [46]:
# import wac 2019 data from LEHD lodes

In [47]:
wac = pd.read_csv('../Raw_Inputs/LEHD/ky_wac_S000_JT00_2019.csv')

In [48]:
# import wac xwalk data from LEHD lodes

In [49]:
geo_xwalk = pd.read_csv('../Raw_Inputs/LEHD/ky_xwalk.csv', low_memory=False)

In [50]:
wac_geo = wac.merge(geo_xwalk, how="left", left_on='w_geocode', right_on='tabblk2020')

In [51]:
wac_geo_grouped = wac_geo.groupby(by = 'trct', as_index = False).sum()

In [52]:
# cleaning and processing

In [53]:
wac_geo_grouped['FOOD_EMP'] = wac_geo_grouped['CNS18'] 

In [54]:
wac_geo_grouped['RETAIL_EMP'] = wac_geo_grouped['CNS07'] 

In [55]:
wac_geo_grouped['OTHER_EMP'] = wac_geo_grouped['CNS01'] + wac_geo_grouped['CNS02'] + wac_geo_grouped['CNS03'] + wac_geo_grouped['CNS04'] + wac_geo_grouped['CNS05'] + wac_geo_grouped['CNS06'] + wac_geo_grouped['CNS08'] + wac_geo_grouped['CNS09'] + wac_geo_grouped['CNS10'] + wac_geo_grouped['CNS11'] + wac_geo_grouped['CNS12'] + wac_geo_grouped['CNS13'] + wac_geo_grouped['CNS14'] + wac_geo_grouped['CNS15'] + wac_geo_grouped['CNS16'] + wac_geo_grouped['CNS17'] + wac_geo_grouped['CNS19'] + wac_geo_grouped['CNS20']

In [56]:
# import shapefile from QGIS (after calculating land area using the shapefile) 

In [57]:
land_area = pd.read_csv('E:/CR2/Repos/TNC-Demand-Model-Southeast/Raw_Inputs/LEHD/census_landarea.csv')

In [58]:
land_area.rename(columns={'GEOID20': 'trct'}, inplace=True)

In [66]:
land_area.tail()

,STATEFP20,COUNTYFP20,TRACTCE20,trct,NAME20,NAMELSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,land_area,merged_fayette_data_CENSUS_TRACT,merged_fayette_data_MEDIAN_AGE,merged_fayette_data_LOW_INC_0,merged_fayette_data_HI_INC_0,merged_fayette_data_LOW_INC_1P,merged_fayette_data_HI_INC_1P,merged_fayette_data_P_BACH_25P,merged_fayette_data_FOOD_EMP,merged_fayette_data_RETAIL_EMP,merged_fayette_data_OTHER_EMP,merged_fayette_data_TOTAL_EMP_DEN,merged_fayette_data_LOGSUM,merged_fayette_data_AIR_F,merged_fayette_data_TOR_F
77,21,67,4205,21067004205,42.05,Census Tract,G5020,S,3689284,16699,37.989952,-84.577491,1.431,21067004205,42.9,0.0,0.0,360.0,180.0,47.007042,0,0,535,373.864430,0,0,0
78,21,67,4207,21067004207,42.07,Census Tract,G5020,S,34955033,106219,38.013885,-84.622867,13.537,21067004207,49.1,0.0,0.0,230.0,85.0,50.110497,193,65,1014,93.964689,0,1,0
79,21,67,4208,21067004208,42.08,Census Tract,G5020,S,8321837,0,38.026559,-84.571985,3.213,21067004208,42.5,25.0,10.0,580.0,165.0,51.132139,1159,626,3059,1507.625272,0,0,0
80,21,67,4209,21067004209,42.09,Census Tract,G5020,S,1676129,0,37.979539,-84.560562,0.647,21067004209,35.2,10.0,0.0,505.0,170.0,43.029016,79,28,83,293.663060,0,0,0
81,21,67,4210,21067004210,42.10,Census Tract,G5020,S,2829042,0,37.973347,-84.542794,1.092,21067004210,33.0,19.0,0.0,480.0,170.0,53.177258,19,298,419,673.992674,0,0,0


In [60]:
# merged 

In [61]:
wac_geo_grouped = pd.merge(wac_geo_grouped, land_area, on='trct', how='inner')

In [62]:
wac_geo_grouped['TOTAL_EMP_DEN'] = (wac_geo_grouped['FOOD_EMP'] + wac_geo_grouped['RETAIL_EMP'] + wac_geo_grouped['OTHER_EMP']) / wac_geo_grouped['land_area']

In [63]:
wac_geo_grouped = wac_geo_grouped[['trct', 'FOOD_EMP', 'RETAIL_EMP', 'OTHER_EMP', 'TOTAL_EMP_DEN']]

In [64]:
den_sum = wac_geo_grouped['TOTAL_EMP_DEN'].sum()
den_sum

214686.21542419965

In [65]:
wac_geo_grouped.head()

,trct,FOOD_EMP,RETAIL_EMP,OTHER_EMP,TOTAL_EMP_DEN
0,21067000101,1465,264,8581,20497.017893
1,21067000102,569,48,1556,7124.590164
2,21067000200,143,120,1032,2133.443163
3,21067000300,9,50,119,425.837321
4,21067000400,23,38,841,1969.432314


In [184]:
wac_geo_grouped.to_csv('../Cleaned_Inputs/Intermediate/wac.csv')